In [1]:
import pandas as pd
import numpy as np
import sys
import os
import xgboost_model

# Data Loading
import ccxt
import pandas as pd

import portfolio

# Technical Indicators 
from datetime import datetime, timedelta
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator, VolumeWeightedAveragePrice, money_flow_index
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import StandardScaler

# MODULE : MODEL LIBRARIES
# all libraries
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.feature_selection import VarianceThreshold
from skopt import BayesSearchCV, gp_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
from skopt.space import Real, Integer
import numpy as np

# Plotting Libraries
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'optuna'

In [2]:
curr_dir = os.getcwd()
file_path = os.path.join(curr_dir, "data/processed_1.csv")
btc_data = pd.read_csv(file_path)

In [4]:
print(btc_data.size)

15147
15114


In [ ]:
print(btc_data.columns)

In [ ]:
data = btc_data

data['timestamp'] = pd.to_datetime(data['timestamp'])
# Features and target
features = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'SMA_5',
       'SMA_10', 'SMA_15', 'EMA_9', 'RSI', 'MACD', 'OBV', 'MFI_10', 'ATR_14',
       'upper_band', 'lower_band', 'daily_momentum', 'rolling_mean_3',
       'rolling_std_3', 'close_lag_1', 'close_lag_2']
targets = ['close_1_ahead', 'close_2_ahead', 'close_5_ahead', 'close_7_ahead', 'close_10_ahead', 'close_15_ahead', 'close_20_ahead', 'close_30_ahead', 'return_1']

#define date range correctly
train_end_date = pd.to_datetime('2025-01-01') 
val_end_date = pd.to_datetime('2025-02-28')
test_end_date = pd.to_datetime('2025-03-01')

# Split the data
train_data = data[data['timestamp'] < train_end_date]
val_data = data[(data['timestamp'] >= train_end_date) & (data['timestamp'] <= val_end_date)]
test_data = data[(data['timestamp'] >= val_end_date)]



# Split into X & y 
X_train = train_data[features].drop(columns = ['timestamp'])
X_val = val_data[features].drop(columns = ['timestamp'])
X_test = test_data[features].drop(columns = ['timestamp'])


y_train = train_data[targets]
y_val = val_data[targets]
y_test = test_data[targets]

print(f"X_train size: {X_train.shape}")
print(f"X_val size: {X_val.shape}")
print(f"X_test size: {X_test.shape}")
print(f"y_train size: {y_train.shape}")
print(f"y_val size: {y_val.shape}")
print(f"y_test size: {y_test.shape}")

In [ ]:
print(data.head())

In [ ]:
final_model_1, y_train_pred_1, y_val_pred_1, y_test_pred_1 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test,'close_1_ahead')
final_model_2, y_train_pred_2, y_val_pred_2, y_test_pred_2 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test, 'close_2_ahead')
final_model_5, y_train_pred_5, y_val_pred_5, y_test_pred_5 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test, 'close_5_ahead')
final_model_10, y_train_pred_10, y_val_pred_10, y_test_pred_10 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test, 'close_10_ahead')
final_model_20, y_train_pred_20, y_val_pred_20, y_test_pred_20 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test, 'close_20_ahead')
final_model_30, y_train_pred_30, y_val_pred_30, y_test_pred_30 = xgboost_model.bayes_xg_boost(X_train, X_val, X_test, y_train, y_val, y_test, 'close_30_ahead')

prediction = final_model_30.predict()

In [ ]:
portfolio = portfolio.Portfolio()
